In [163]:
import pandas as pd
import os
%xmode minimal
pd.set_option("mode.copy_on_write", True)

Exception reporting mode: Minimal


# 9. Tidy data
### 9.1. Billboard

In [201]:
billbo = pd.read_csv("../DsKi_DataPreparation/data/Aufgaben/billboard.csv")

In [202]:
billbo.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
# Berechnen der Wochen in den Top 100
week_columns = [col for col in billbo.columns if col.startswith('wk')]
billbo['Weeks'] = billbo[week_columns].notna().sum(axis=1)

# Umformen des DataFrames in ein tidy format
tidy_billboard = billbo.melt(id_vars=["year", "artist", "track", "time", "date.entered", "Weeks"])

# Bereinigen der Woche und Umbenennen der Spalten
tidy_billboard.rename(columns={"date.entered": "date_entered"}, inplace=True)

# Optional: Berechnen der neuen Datumsspalte
tidy_billboard["date"] = pd.to_datetime(tidy_billboard["date_entered"]) + \
                         pd.to_timedelta(tidy_billboard["Weeks"] - 1, unit='w')

In [204]:
tidy_billboard = tidy_billboard[['year', 'artist', 'track', 'time', 'date_entered', 'Weeks', 'date']]


In [205]:
tidy_billboard.head()

,year,artist,track,time,date_entered,Weeks,date
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,7,2000-04-08
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,3,2000-09-16
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,53,2001-04-07
3,2000,3 Doors Down,Loser,4:24,2000-10-21,20,2001-03-03
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,18,2000-08-12


In [206]:
data = {
"name" : ["Philipp Woods", "Philipp Woods", "Philipp Woods",
"Jessica Cordero", "Jessica Cordero"],
"property" : ["age", "height", "age", "age", "height"],
"values" : [45, 186, 50, 37, 156]
}
df = pd.DataFrame(data)
df

,name,property,values
0,Philipp Woods,age,45
1,Philipp Woods,height,186
2,Philipp Woods,age,50
3,Jessica Cordero,age,37
4,Jessica Cordero,height,156


a) Formen Sie nach tidy data um. Welches Problem entsteht? <br>
b) Wie kann man das lösen? <br>
c) Wie könnte man eine Spalte ergänzen, so dass die Werte eindeutig identifiziert
werden?

In [209]:
# Schritt 2: Hinzufügen einer Spalte zur eindeutigen Identifizierung
df['id'] = df.groupby(['name', 'property']).cumcount()

# Schritt 3: Umformung unter Beibehaltung der Eindeutigkeit
tidy_df_unique = df.pivot_table(index=['name', 'id'], columns='property', values='values').reset_index()
tidy_df_unique.columns.name = None  # Entfernt den Namen der Spalten-Gruppe
tidy_df_unique


,name,id,age,height
0,Jessica Cordero,0,37.0,156.0
1,Philipp Woods,0,45.0,186.0
2,Philipp Woods,1,50.0,NaN


Alternativ

a)

In [213]:
df.pivot(index=["name"], columns = 'property', values = 'values')

ValueError: Index contains duplicate entries, cannot reshape

b)

In [214]:
df.pivot_table(index=["name"], columns = 'property', values = 'values')

property,age,height
name,,
Jessica Cordero,37.0,156.0
Philipp Woods,47.5,186.0


9.3 Wandeln Sie nach tidy data. Was sind die Variablen?

In [216]:
# Zuerst erstellen wir einen DataFrame basierend auf den Informationen aus dem Bild.
# Da wir den Inhalt des Bildes nicht direkt auslesen können, müssen wir die Daten manuell eingeben.
data = {
    "pregnant": ["yes", "no"],
    "male": [None, 20],
    "female": [10, 12]
}

# DataFrame erstellen
wide_df = pd.DataFrame(data)

# Umwandlung in Tidy Data Format
tidy_df = wide_df.melt(id_vars=["pregnant"], value_vars=["male", "female"], 
                       var_name="gender", value_name="count")
tidy_df

,pregnant,gender,count
0,yes,male,NaN
1,no,male,20.0
2,yes,female,10.0
3,no,female,12.0


Beispiel Tidy Data

In [172]:
data = {
    "Stadt": ["Berlin", "Hamburg"],
    "Tag1": [20, 18],
    "Tag2": [22, 19]
}
df = pd.DataFrame(data)

# Umwandeln in Tidy Data
tidy_df = df.melt(id_vars=["Stadt"], value_vars=["Tag1", "Tag2"],
                  var_name="Tag", value_name="Temperatur")

tidy_df

,Stadt,Tag,Temperatur
0,Berlin,Tag1,20
1,Hamburg,Tag1,18
2,Berlin,Tag2,22
3,Hamburg,Tag2,19


In [173]:
data = {
    "Produkt": ["Apfel", "Banane"],
    "Q1": [300, 200],
    "Q2": [250, 220]
}
df = pd.DataFrame(data)

# Umwandeln in Tidy Data
tidy_df = df.melt(id_vars=["Produkt"], value_vars=["Q1", "Q2"],
                  var_name="Quartal", value_name="Verkaufszahlen")
tidy_df


,Produkt,Quartal,Verkaufszahlen
0,Apfel,Q1,300
1,Banane,Q1,200
2,Apfel,Q2,250
3,Banane,Q2,220


### 9.4. Ebola
Untersuchen Sie die Datei country_timeseries.csv und wandeln Sie in Tidy-Form. Berechnen
Sie auch die Todesrate (Deaths je Cases).

In [218]:
country_df = pd.read_csv("../DsKi_DataPreparation/data/Aufgaben/country_timeseries.csv")

In [219]:
country_df.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [220]:
country_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Date', 'Day', 'Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates', 'Cases_Spain',
       'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia', 'Deaths_SierraLeone',
       'Deaths_Nigeria', 'Deaths_Senegal', 'Deaths_UnitedStates',
       'Deaths_Spain', 'Deaths_Mali'],
      dtype='object')>

In [223]:
# Step 1: Melting the data to have tidy data with 'Country' and 'Case' as separate columns

# Melting cases columns
cases_melted = pd.melt(country_df, id_vars=['Date', 'Day'], value_vars=[col for col in country_df.columns if 'Cases_' in col],
                       var_name='Country_Type', value_name='Cases')

# Melting deaths columns
deaths_melted = pd.melt(country_df, id_vars=['Date', 'Day'], value_vars=[col for col in country_df.columns if 'Deaths_' in col],
                        var_name='Country_Type', value_name='Deaths')

# Step 2: Renaming the 'Country_Type' column to extract just the country names for both dataframes
cases_melted['Country'] = cases_melted['Country_Type'].str.replace('Cases_', '')
deaths_melted['Country'] = deaths_melted['Country_Type'].str.replace('Deaths_', '')

# Step 3: Merging the cases and deaths dataframe on 'Country', 'Date', and 'Day'
combined_data = pd.merge(cases_melted[['Date', 'Day', 'Country', 'Cases']],
                         deaths_melted[['Date', 'Day', 'Country', 'Deaths']],
                         on=['Country', 'Date', 'Day'],
                         how='outer')

# Step 4: Calculating the death rate
combined_data['Death_Rate'] = combined_data['Deaths'] / combined_data['Cases']

# Step 5: Cleaning the final dataframe
final_tidy_data = combined_data[['Date', 'Day', 'Country', 'Cases', 'Deaths', 'Death_Rate']]

# Displaying the first few rows of the final tidy dataframe
final_tidy_data.head(5)


,Date,Day,Country,Cases,Deaths,Death_Rate
0,1/5/2015,289,Guinea,2776.0,1786.0,0.643372
1,1/4/2015,288,Guinea,2775.0,1781.0,0.641802
2,1/3/2015,287,Guinea,2769.0,1767.0,0.638137
3,1/2/2015,286,Guinea,NaN,NaN,NaN
4,12/31/2014,284,Guinea,2730.0,1739.0,0.636996


In [224]:
tidy_df.head()

,pregnant,gender,count
0,yes,male,NaN
1,no,male,20.0
2,yes,female,10.0
3,no,female,12.0


### 9.5. Weather
Untersuchen Sie weather.csv. Es sind einige Wetterdaten (nur) einer Wetterstation
MX17004 in Mexiko.

In [225]:
# Load the weather data from the uploaded CSV file
weather_file_path = "./data/Aufgaben/weather.csv"
weather_data = pd.read_csv(weather_file_path)

# Display the first few rows to understand its structure
weather_data.head()


,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


a) Bringen Sie das Wetter in tidy-Form. Hier tauchen mehrere Probleme gleichzeitig auf.


In [227]:
# Teil a) Erneute Umwandlung der Wetterdaten in Tidy-Form

# Melting the dataframe to long format with all days included
weather_long = pd.melt(weather_data, 
                       id_vars=['id', 'year', 'month', 'element'], 
                       var_name='day', 
                       value_name='temp')

# Extracting day number and converting to integer
weather_long['day'] = weather_long['day'].str.extract('(\d+)').astype(int)

# Creating two separate dataframes for tmax and tmin
weather_tmax = weather_long[weather_long['element'] == 'tmax'].drop('element', axis=1)
weather_tmin = weather_long[weather_long['element'] == 'tmin'].drop('element', axis=1)

# Merging the tmax and tmin dataframes on 'id', 'year', 'month', and 'day'
weather_tidy = pd.merge(weather_tmax, weather_tmin, on=['id', 'year', 'month', 'day'], how='outer')

# Sorting the dataframe by date to make it easier to investigate what happens at the end of February
weather_tidy = weather_tidy.sort_values(by=['year', 'month', 'day'])

# Displaying the first few rows of the tidy dataframe
weather_tidy.shape


(341, 6)

b) Was ist mit den fehlenden Werten? Was passiert Ende Februar?

In [181]:
feb_weather = weather_tidy.loc[(weather_tidy["month"]==2 )& (weather_tidy["day"]>28)]
feb_weather.head(10)

,id,year,month,day,temp_x,temp_y
309,MX17004,2010,2,29,NaN,NaN
320,MX17004,2010,2,30,NaN,NaN
331,MX17004,2010,2,31,NaN,NaN


Beim Betrachten der Daten am Ende Februar stellen wir fest, dass es Zeilen für den 29., 30. und 31. Februar gibt, die alle NaN-Werte enthalten. Da der Februar maximal 29 Tage hat (in einem Schaltjahr), sind diese Einträge offensichtlich ungültig und sollten entfernt werden.

c) Erzeugen Sie eine „date“ Spalte mit dem Datum im ISO-Format („2010-01-07“)

In [182]:
def create_date(row):
    try:
        return pd.Timestamp(year=row['year'], month=row['month'], day=row['day'])
    except ValueError:
        return pd.NaT

# Anwenden der Funktion auf jede Zeile
weather_tidy['date'] = weather_tidy.apply(create_date, axis=1)

# Anzeige der ersten Zeilen mit der neuen 'date' Spalte
weather_tidy.head()


,id,year,month,day,temp_x,temp_y,date
0,MX17004,2010,1,1,NaN,NaN,2010-01-01
11,MX17004,2010,1,2,NaN,NaN,2010-01-02
22,MX17004,2010,1,3,NaN,NaN,2010-01-03
33,MX17004,2010,1,4,NaN,NaN,2010-01-04
44,MX17004,2010,1,5,NaN,NaN,2010-01-05


### 9.6. CORDIS

In [228]:
projects_path = "./data/cordis_eu_2020/project.csv"
org_path = "./data/cordis_eu_2020/organization.csv"

In [229]:
eu_proj = pd.read_csv(projects_path, sep=';', decimal=',')
eu_org = pd.read_csv(org_path, sep=';', decimal=',')

In [230]:
eu_proj.head(1)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
0,888561,Mag-Spin,CLOSED,A novel material for watch crystals and watch ...,2019-11-01,2020-04-30,71429.0,50000.0,H2020-EU.2.3.,EIC-SMEInst-2018-2020,2019-12-04,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,Watch companies over the last century have con...,2022-08-17 14:28:09,226627,10.3030/888561


In [231]:
# Angenommen df ist dein DataFrame und 'iso_date' die Spalte mit den ISO-Daten.
eu_proj['start_year'] = pd.to_datetime(eu_proj['startDate']).dt.year.astype('Int64')

eu_proj['end_year'] = pd.to_datetime(eu_proj['endDate']).dt.year.astype('Int64')


In [232]:
def create_year_list(start, end):
    # Prüfen, ob start und end gültige Zahlen sind
    if pd.notna(start) and pd.notna(end):
        return list(range(int(start), int(end) + 1))
    else:
        return []

# Anwenden der Funktion auf jede Zeile
eu_proj['years'] = eu_proj.apply(lambda row: create_year_list(row['start_year'], row['end_year']), axis=1)



In [233]:
eu_proj.sample(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,start_year,end_year,years
7398,729366,E-ProMS,CLOSED,Business-to-Business (B2B) Business-to-Govern...,2016-06-01,2016-11-30,71429.0,50000.0,H2020-EU.2.1.1.,SMEInst-01-2016-2017,...,H2020-SMEINST-1-2016-2017,SME-1,NaN,Procurement processes and supplier relations a...,2022-08-12 09:04:29,204402,10.3030/729366,2016,2016,[2016]
3973,810752,FoodEnTwin,CLOSED,Twinning of research activities for the fronti...,2018-09-01,2021-12-31,998012.5,998012.5,H2020-EU.4.b.,WIDESPREAD-05-2017,...,H2020-WIDESPREAD-05-2017-Twinning,CSA,NaN,The objective of FoodEnTwin is to create a net...,2022-10-28 13:57:10,217831,10.3030/810752,2018,2021,"[2018, 2019, 2020, 2021]"
5055,800604,BC AMEL,TERMINATED,BEFORE CONSUMPTION: AMBIGUOUS MORALITIES AND ...,2018-08-01,2020-07-31,195454.8,195454.8,H2020-EU.1.3.,MSCA-IF-2017,...,H2020-MSCA-IF-2017,MSCA-IF,NaN,Irresponsible breeding of domestic animals is ...,2023-07-23 19:44:48,214323,10.3030/800604,2018,2020,"[2018, 2019, 2020]"
